In [21]:
## Bibliotecas
import yfinance
import plotly_express as px
import plotly.graph_objects as go
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [20]:
# Ativos: WEGE3, PETR4 e CEAB3
lista_ativo = ['WEGE3','PETR4','CEAB3']
# Primeiro montar gráfico de candlestick 
# Depois fazer scraping e pegar informações do Brazil Journal

In [ ]:
# Desenvolvimento do Gráfico

# Carregando dados da yfinance
df = yfinance.download(lista_ativo[1] + '.SA',
                       start='2023-01-01',
                       )
df['Adj Close'] = df['Adj Close'] - 0.001
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'])
                     ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

#Fonte:https://plotly.com/python/candlestick-charts/

In [24]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-02,23.540001,23.809999,22.799999,22.920000,17.760120,78424700
2023-01-03,22.940001,23.100000,22.129999,22.340000,17.310694,96750300
2023-01-04,21.959999,23.590000,21.830000,23.049999,17.860853,129504000
2023-01-05,23.340000,24.040001,23.150000,23.879999,18.504000,73886000
2023-01-06,23.940001,24.320000,23.549999,23.740000,18.395517,51851500
...,...,...,...,...,...,...
2023-12-28,37.230000,37.360001,37.040001,37.240002,37.240002,21421900
2024-01-02,37.439999,37.889999,37.400002,37.779999,37.779999,24043800
2024-01-03,37.799999,39.119999,37.669998,38.959999,38.959999,52300200


In [28]:
# Scraping
#Link https://braziljournal.com/?s={nome ativo/nome empresa}

# A princípio vamos botar WEG, uma vez que WEGE só retorna uma pesquisa


#Request
url = 'https://braziljournal.com/?s=weg'

r = requests.get(url)

#Scraping dos dados
soup = BeautifulSoup(r.text,'html.parser')


In [75]:
todas_noticiais = soup.find_all(class_='wrap-bj')
lista_titulo = []
lista_link = []
lista_categoria = []
for i in todas_noticiais:
    elementos_h2 = i.find_all('h2', class_='boxarticle-infos-title')[:3]
    elemento_p = i.find_all('p', class_='boxarticle-infos-tag')[:3]
   # Agora você pode trabalhar com os elementos h2 e p encontrados
  
    for elemento_h2, elemento_p in zip(elementos_h2, elemento_p):
  
        
        lista_titulo.append(elemento_h2.a.text.strip())
        lista_link.append(elemento_h2.a['href'])
        lista_categoria.append(elemento_p.a.text.strip())

        # Imprime o texto dentro da tag <a> dentro de 'p'
        print(elemento_p.a.text.strip())
        # Imprime o valor do atributo 'href' da tag <a> dentro de 'p'    
        # Imprime o texto dentro da tag <a> dentro de 'h2'
        print(elemento_h2.a.text.strip())
        # Imprime o valor do atributo 'href' da tag <a> dentro de 'h2'
        print(elemento_h2.a['href'])
        

Estratégia corporativa
WEG faz sua maior aquisição; investe R$ 2 bi para crescer em motores e geradores
https://braziljournal.com/weg-faz-sua-maior-aquisicao-investe-r-2-bi-para-crescer-em-motores-e-geradores/
WEG
WEG faz parcerias de R$ 1,4 bi em autogeração eólica
https://braziljournal.com/weg-faz-parcerias-de-r-14-bi-em-autogeracao-eolica/
Research
WEG leva um (raro) downgrade no Itaú
https://braziljournal.com/weg-leva-um-raro-downgrade-no-itau/


In [76]:
print(f'Categoria:{lista_categoria[0]}')
print(f'Titulo:{lista_titulo[0]}')
print(f'Link:{lista_link[0]}')

Categoria:Estratégia corporativa
Titulo:WEG faz sua maior aquisição; investe R$ 2 bi para crescer em motores e geradores
Link:https://braziljournal.com/weg-faz-sua-maior-aquisicao-investe-r-2-bi-para-crescer-em-motores-e-geradores/


In [19]:
# Criando função de Scraping da Notícias
def scraping_noticia(stock_name,qntd_noticias = 3):
    #Request
    url = f'https://braziljournal.com/?s={stock_name}'

    r = requests.get(url)

    #Scraping dos dados
    soup = BeautifulSoup(r.text,'html.parser')
    
    #Div do html que contém todas as noticias
    todas_noticias = soup.find_all(class_='wrap-bj')

    #Dicionário para armazenar as informações:
    dict_url = {
        'titulo' : [],
        'link' : [],
        'categoria' : []
    }
    
    #Looping para interagir com cada notícia
    for i in todas_noticias:
        
        # Titulo e url    
        elementos_titulo = i.find_all('h2', class_='boxarticle-infos-title')[:qntd_noticias]
        
        # Tópico da noticia 
        elemento_categoria = i.find_all('p', class_='boxarticle-infos-tag')[:qntd_noticias]
    
        # Looping com os elementos para pegar informações de cada noticia
        for elementos_titulo, elemento_categoria in zip(elementos_titulo, elemento_categoria):
            dict_url['titulo'].append(elementos_titulo.a.text.strip())
            dict_url['link'].append(elementos_titulo.a['href'])
            dict_url['categoria'].append(elemento_categoria.a.text.strip())
    return dict_url

In [22]:
resultados = scraping_noticia('C%26A')

In [23]:
resultados

{'titulo': ['C&A surpreende ao dobrar lucro e elevar Ebitda; ações disparam 16%',
  'C&A dispara pós-balanço; controle de caixa será crucial para 2023, diz CEO',
  'A C&A está barata e a estratégia, correta, diz a XP.  Já o macro…'],
 'link': ['https://braziljournal.com/play/c-acoes-disparam-13/',
  'https://braziljournal.com/play/c-controle-de-caixa-sera-crucial-para-2023-diz-ceo/',
  'https://braziljournal.com/a-ca-esta-barata-e-a-estrategia-correta-diz-a-xp-ja-o-macro/'],
 'categoria': ['Varejo & Consumo', 'Varejo & Consumo', 'Research']}

In [86]:
options_stock = ['WEGE3.SA','PETR4.SA','CEAB3.SA']
simbolo_pesquisa = ['WEG','petrobras%20ação','C%26A']


In [87]:
options_stock.index('WEGE3.SA')

0

In [88]:
from scraping import scraping_noticia

In [89]:
stock = 'WEGE3.SA'
pos = options_stock.index(f'{stock}')
palavra_chave = simbolo_pesquisa[pos]
dict_resultado = scraping_noticia(palavra_chave)


## Fazendo Scraping do TEXTO das Notícias

In [ ]:
# Se tiver
# /play/ -> boxarticle-infos-text
# c.c -> 

In [42]:
lista_link = resultados['link']
#Microuniverso para depois replicar e ver resultado
r = requests.get(lista_link[2])

soup = BeautifulSoup(r.text,'html.parser')

In [43]:
noticia = soup.find_all(class_='boxarticle-infos-text')[0]
paragrafos = noticia.find_all('p')
for parag in paragrafos:
    print(parag.text)

In [ ]:
noticia = soup.find_all(class_='post-content-text')[0]
paragrafos = noticia.find_all('p')
for parag in paragrafos:
    print(parag.text)

## Analise de Sentimento das Noticias
##### OBS: Tem que passar todo o texto para inglês. Ou seja, devemos utilizar alguma api de tradução de texto se quisermos fazer algo automático.

In [1]:
## Textos a serem analisados:
#OBS texto_analyze foi traduzido pelo chat gpt...
texto_analyze_1 = 'A WEG está comprando a divisão de motores industriais e geradores da Regal Rexnord — o maior M&A de sua história, e uma transação que vai aumentar a presença internacional da companhia brasileira em 20% com marcas fortes e de nicho.  A WEG está avaliando os ativos da Regal — uma empresa americana listada na NYSE com market cap de US$ 9,6 bilhões — a um enterprise value de US$ 400 milhões. O pagamento será feito em dinheiro assim que a companhia obtiver as aprovações regulatórias, o que está previsto para o início do ano que vem.  A transação não deve ter um impacto relevante na estrutura de capital da WEG, que após o desembolso continuará com uma posição de caixa líquido de R$ 1 bi. O ‘carve out’ da Regal inclui 10 fábricas em sete países (EUA, México, China, Índia, Itália, Holanda e Canadá), além de marcas fortes e um time de cerca de 2.800 funcionários. A WEG disse que a distribuição geográfica dos ativos é complementar à sua, e vai ajudar a aumentar a escala do negócio e gerar eficiências de custo.  A maior parte da receita da operação adquirida vem da fabricação de motores industriais de baixa tensão (75%), e o restante de geradores elétricos. “São três marcas muito tradicionais nos Estados Unidos e na Europa, e com bastante aderência com o cliente final,” Alberto Kuba, o diretor da WEG Motores Industriais, disse numa call com analistas agora há pouco.  Segundo ele, as marcas Cemp e Rotor têm operações “bem interessantes” de motores de nicho nos Estados Unidos e Europa — e a transação vai “ajudar a entrar em mercados que são mais difíceis, como o de óleo e gás e naval.”  “São mercados em que a WEG já tem os produtos, mas como eles dependem de muitas homologações locais, o crescimento é muito lento. A Regal já fez esse movimento anos atrás, ganhando esses mercados que são de difícil penetração.” Segundo o executivo, há um overlap muito baixo de clientes entre as duas operações de motores industriais. Na Índia, por exemplo, a WEG entrou recentemente com uma linha de combate de motores de uso geral. “Com a aquisição vamos ter um portfólio completo de motores e totalmente complementar,” disse ele.  Os ativos da Regal que estão sendo comprados pela WEG tiveram receita líquida de US$ 541 milhões no ano passado, e EBITDA de US$ 51 milhões (margem de 9,5%). Para efeito de comparação, a WEG fez uma receita de US$ 6 bilhões no mesmo período, e opera com uma margem na casa dos 18,7%.  O valor pago pela WEG implica um múltiplo de 7,7x EV/EBITDA do ano passado, em comparação aos 20x que a WEG negocia hoje na Bolsa.  O JP Morgan disse que a aquisição é positiva porque mostra a “capacidade da WEG de continuar crescendo, ainda que inorganicamente, e traz sinergias potenciais importantes (tanto comerciais quanto técnicas e de custo).” O banco nota que, no curto prazo, a transação terá um impacto negativo de 0,8 ponto percentual na margem EBITDA da WEG, dada a margem muito menor da Regal. “Mas vemos um impacto agregado positivo de entre 3% e 4%,” escreveram os analistas.  O CFO da WEG, André Luis Rodrigues, disse que a companhia espera levar a margem EBITDA das operações da Regal para um patamar próximo às margens que a WEG tem neste países e mercados.  Essa melhora nas margens virá das sinergias — com a integração dos negócios — e também de um uso maior da capacidade da Regal. Hoje, as 10 fábricas que estão sendo adquiridas operam com uma ociosidade de cerca de 50%, e a ideia da WEG é levá-las a 100%.  “Teoricamente, temos condições de dobrar o volume produzido hoje apenas com a estrutura que já existe, sem fazer nenhum grande investimento adicional,” disse Kuba.  O executivo ressaltou ainda que há uma oportunidade grande de verticalização de parte dessas operações, com a fundição que a WEG tem no México passando a fornecer produtos para as fábricas da Regal nos EUA e México.  Segundo ele, a fundição no México tem capacidade ociosa que pode ser usada para atender essas novas fábricas, sem precisar de mais capex.  Para a vertical de geradores da WEG, a transação de hoje é um “game changer,” nas palavras de João Paulo da Silva, o diretor da WEG Energia.  “A WEG tem uma posição de liderança em geradores no Brasil, mas no exterior nossa posição era muito pequena. Temos poucos clientes fora do Brasil,” disse ele. “A aquisição da Marathon — que é uma marca consolidada no Canadá e EUA — vai nos posicionar num novo patamar em termos de market share. Vamos passar a ser um player importante no mundo, principalmente na América do Norte.” Assim como em motores industriais, a fábrica de geradores da Regal também estava largamente ociosa. Enquanto em motores a ociosidade tinha a ver com uma falta de foco da Regal nesse segmento, em geradores isso aconteceu porque a companhia construiu recentemente novas fábricas na China e México, com uma capacidade muito maior.'
texto_1_chat_gpt = """
WEG is acquiring the industrial motors and generators division of Regal Rexnord — the largest M&A deal in its history, and a transaction that will increase the Brazilian company's international presence by 20% with strong and niche brands. WEG is evaluating Regal's assets — an American company listed on the NYSE with a market cap of $9.6 billion — at an enterprise value of $400 million. The payment will be made in cash once the company obtains regulatory approvals, expected in the beginning of next year.

The transaction is not expected to have a significant impact on WEG's capital structure, which will maintain a net cash position of R$1 billion after the disbursement. The carve-out from Regal includes 10 factories in seven countries (USA, Mexico, China, India, Italy, Netherlands, and Canada), along with strong brands and a team of about 2,800 employees. WEG stated that the geographical distribution of the acquired assets is complementary to its own, helping to increase business scale and generate cost efficiencies.

The majority of the acquired operation's revenue comes from the manufacturing of low-voltage industrial motors (75%), with the remainder from electrical generators. Alberto Kuba, the director of WEG Industrial Motors, mentioned in a recent analyst call that the Cemp and Rotor brands have "very interesting" niche motor operations in the USA and Europe. The transaction will help WEG enter markets that are more challenging, such as oil and gas and naval.

Kuba highlighted that there is very low customer overlap between the two industrial motor operations. In India, for example, WEG recently entered with a line of general-purpose motors. "With the acquisition, we will have a complete and totally complementary portfolio," he said.

The Regal assets being acquired by WEG had a net revenue of $541 million last year, with an EBITDA of $51 million (9.5% margin). For comparison, WEG had a revenue of $6 billion in the same period, operating with a margin around 18.7%. The amount paid by WEG implies a multiple of 7.7x EV/EBITDA for the last year, compared to the 20x that WEG is currently trading on the stock exchange.

JP Morgan stated that the acquisition is positive because it shows WEG's "ability to continue growing, even inorganically, and brings potential important synergies (both commercial and technical and cost)." The bank noted that, in the short term, the transaction will have a negative impact of 0.8 percentage points on WEG's EBITDA margin, given the much lower margin of Regal. "But we see an aggregate positive impact of between 3% and 4%," the analysts wrote.

WEG's CFO, André Luis Rodrigues, said the company expects to bring the EBITDA margin of Regal's operations to a level close to the margins that WEG has in these countries and markets. This margin improvement will come from synergies with business integration and a greater use of Regal's capacity. Currently, the 10 acquired factories operate with an idleness of about 50%, and WEG's idea is to bring them to 100%. "Theoretically, we have the conditions to double the volume produced today with the existing structure, without making any major additional investment," said Kuba.

The executive also emphasized that there is a significant opportunity for vertical integration of part of these operations, with the foundry that WEG has in Mexico supplying products to Regal's factories in the USA and Mexico. According to him, the foundry in Mexico has idle capacity that can be used to serve these new factories, without the need for additional capex.

For WEG's generator division, today's transaction is a "game changer," in the words of João Paulo da Silva, director of WEG Energia. "WEG has a leadership position in generators in Brazil, but abroad our position was very small. We have few customers outside Brazil," he said. "The acquisition of Marathon — which is a consolidated brand in Canada and the USA — will position us at a new level in terms of market share. We will become a significant player worldwide, especially in North America." Similar to industrial motors, Regal's generator factory was also largely idle. While in motors, idleness was due to a lack of focus by Regal in that segment, in generators, this happened because the company recently built new factories in China and Mexico with much higher capacity.

"""
texto_analyze_2 = 'A WEG fechou duas parcerias para a autoprodução de energia eólica — um movimento que deve ajudar a gigante industrial a reduzir seus custos com eletricidade ao mesmo tempo em que endereça suas metas de sustentabilidade. A primeira parceria foi feita com a Alupar.  A WEG virou sócia da companhia de transmissão e geração num parque eólico no Rio Grande do Norte, o AW Santa Régia. Esse parque tem capacidade de 37,8 MW e vai começar a operar em janeiro do ano que vem. A WEG fechou um contrato de compra de energia de 18 anos no valor de R$ 460 milhões. O parque vai entregar cerca de 15 MW de energia. O segundo acordo foi feito com a 2W Ecobank, e envolve os parques eólicos Anemus I, II e III, também no Rio Grande do Norte. A WEG fechou um contrato de 20 anos no valor de R$ 970 milhões; os três parques vão entregar 30 MW de energia para a companhia.  Daniel Godinho, o diretor de sustentabilidade e relações institucionais da WEG, disse ao Brazil Journal que os dois projetos vão levar para 90% o uso de energias renováveis nas fábricas da companhia no Brasil. “Também estamos avançando em projetos nas nossas operações no exterior. Na China e na Índia já vamos atingir 50% e 90% de energia renovável, por exemplo,” disse ele. “Com todos esses movimentos, o grupo WEG vai atingir globalmente 75% de consumo de energia com fontes renováveis, em comparação aos 15% de hoje.” A Weg tem 52 fábricas em 15 países e o objetivo da companhia é que 100% de toda a energia usada por elas venha de fontes renováveis no modelo de autogeração.  Segundo Daniel, as parcerias de hoje vão ao encontro das metas de sustentabilidade da WEG, que se comprometeu no ano passado a reduzir em 52% suas emissões de CO2 até 2030 e a chegar à net zero até 2050. Os investimentos, no entanto, também devem ajudar a companhia a reduzir seus custos com energia, como notou o BTG Pactual. “Temos uma visão positiva da iniciativa porque ela visa otimizar os custos de energia em suas fábricas para aumentar a eficiência operacional,” escreveu o analista Lucas Marquiori. “Os custos com energia são um elemento significativo das fábricas da WEG, e as economias que virão com esses projetos podem contribuir para uma geração de valor adicional no longo prazo.” Daniel, da WEG, também disse que um aspecto interessante das duas parcerias é “que fomos capazes de usar nossa própria tecnologia nos projetos.” Tanto o parque da Alupar quanto o da 2w usam aerogeradores produzidos pela WEG. No total, os parques terão 42 aerogeradores.'
texto_analyze_3 = 'WEG faz parcerias de R$ 1,4 bi em autogeração eólica'


In [2]:
# Traduzindo os textos
from googletrans import Translator
trans = Translator()
lista_texto_traduzido = []
for i in range(1,4):
    texto = globals()[f'texto_analyze_{i}']
    texto_traduzido = trans.translate(texto,dest='en').text
    lista_texto_traduzido.append(texto_traduzido)



A WEG está comprando a divisão de motores industriais e geradores da Regal Rexnord — o maior M&A de sua história, e uma transação que vai aumentar a presença internacional da companhia brasileira em 20% com marcas fortes e de nicho.  A WEG está avaliando os ativos da Regal — uma empresa americana listada na NYSE com market cap de US$ 9,6 bilhões — a um enterprise value de US$ 400 milhões. O pagamento será feito em dinheiro assim que a companhia obtiver as aprovações regulatórias, o que está previsto para o início do ano que vem.  A transação não deve ter um impacto relevante na estrutura de capital da WEG, que após o desembolso continuará com uma posição de caixa líquido de R$ 1 bi. O ‘carve out’ da Regal inclui 10 fábricas em sete países (EUA, México, China, Índia, Itália, Holanda e Canadá), além de marcas fortes e um time de cerca de 2.800 funcionários. A WEG disse que a distribuição geográfica dos ativos é complementar à sua, e vai ajudar a aumentar a escala do negócio e gerar efici

In [7]:
for texto_traduzido in lista_texto_traduzido:
    print(texto_traduzido)

Weg is buying the industrial engines and generators of Regal Rexnord - the largest M&A in its history, and a transaction that will increase the international presence of the Brazilian company by 20% with strong and niche brands.Weg is valuing Regal's assets - an American company listed on NYSE with Market Cap of $ 9.6 billion - to an Enterprise Value of $ 400 million.Payment will be made in cash as soon as the company obtains regulatory approvals, which is scheduled for the beginning next year.The transaction should not have a relevant impact on WEG's capital structure, which after the disbursement will continue with a net cash position of $ 1 billion.Regal's Carve Out includes 10 factories in seven countries (US, Mexico, China, India, Italy, the Netherlands and Canada), as well as strong brands and a team of about 2,800 employees.Weg said the geographical distribution of assets is complementary to yours, and will help increase business scale and generate cost efficiencies.Most of the 

### FinBert

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

#fonte: https://huggingface.co/yiyanghkust/finbert-tone

/home/bernardo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-08 00:24:29.677995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-08 00:24:29.704935: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 00:24:29.704964: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 00:24:29.704998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when on

### NTLK


In [12]:
import nltk
### Uncomment it when the script runs for the first time 
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()



#  Podemos acrescentar pesos para palavras no modelo:
# new_words = {
#     'crushes': 10,
#     'bate': 5,
#     'problema': -10,
#     'queda': -100,
# }
# sia.lexicon.update(new_words)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/bernardo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:

sentiment = sia.polarity_scores(texto_analyze_1)
sentiment_2 = sia.polarity_scores(texto_analyze_2)
sentiment_3 = sia.polarity_scores(texto_analyze_3)

print('noticia 1', sentiment)
print('noticia 2', sentiment_2)
print('noticia 3', sentiment_3)
for texto_traduzido in lista_texto_traduzido:
    print(sia.polarity_scores(texto_traduzido))

noticia 1 {'neg': 0.026, 'neu': 0.959, 'pos': 0.014, 'compound': -0.7066}
noticia 2 {'neg': 0.059, 'neu': 0.941, 'pos': 0.0, 'compound': -0.9595}
noticia 3 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.031, 'neu': 0.892, 'pos': 0.077, 'compound': 0.9821}
{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'compound': 0.9887}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


### Api GPT4 para Análise Sentimento (Só dá para usar o pago)

In [10]:
import openai
with open('./api_key_chatgpt.txt', 'r') as arquivo:
    api_key = arquivo.read()

#Arguemntos para call da api:
openai.api_key = api_key
model = 'text-davinci=003' 
texto_to_analyze = 'WEG faz sua maior aquisição; investe R$ 2 bi para crescer em motores e geradores'
completion = openai.completions.create(model=model,
                                      prompt=(f"Sentiment analysis of the following text: '{texto_to_analyze}' \n\nSentiment Score: "),
                                      max_tokens=1,
                                      temperature=0
                                      )

#Fonte: https://www.youtube.com/watch?v=Vps-0LEa9SA&t=366s